In [ ]:
import json
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
from numpy import linalg as LA

import pandas as pd

import networkx as nx
import community

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.metrics import adjusted_rand_score

import pickle

In [ ]:
baseline = "baseline_720_24_1dot02.pkl" 
thresh = "thresh_720_24_0dot3_1.pkl"
thresh2 = "threshold2_720_24_1dot05_3.pkl"

with open("../data/processed/"+baseline, 'rb') as f:
    baseline_clusters = pickle.load(f)
    
with open("../data/processed/"+thresh, 'rb') as f:
    thresh_clusters = pickle.load(f)

with open("../data/processed/"+thresh2, 'rb') as f:
    thresh2_clusters = pickle.load(f)

In [ ]:
clustering_list = ["baseline_clusters", "thresh2_clusters"]

In [ ]:
all_clustering = {"baseline_clusters":baseline_clusters, "thresh2_clusters": thresh2_clusters}

## Stability

In [ ]:
def compute_rand_score(clusters):
    nb_clusters = len(clusters.keys())
    rand_score = []
    for i in range(nb_clusters - 1):
        score = adjusted_rand_score(list(clusters[list(clusters.keys())[i]].values()), list(clusters[list(clusters.keys())[i+1]].values()))
        rand_score.append(score)
    return rand_score

In [ ]:
def compute_nb_clusters(clusters):
    nb_clusters = len(clusters.keys())
    list_nb_clusters = []
    for i in range(nb_clusters - 1):
        nb = max(clusters[list(clusters.keys())[i]].values())
        list_nb_clusters.append(nb)
    return list_nb_clusters

In [ ]:
clustering_sizes = {}
for i in range(len(clustering_list)):
    clustering_sizes[clustering_list[i]] = compute_nb_clusters(all_clustering[clustering_list[i]])

In [ ]:
# Combine the data in a single dataframe
df_plot = pd.DataFrame({'Group': [c for c in clustering_list for size in clustering_sizes[c]],
                   'Value': [size for c in clustering_list for size in clustering_sizes[c]]})

# Create a boxplot with Seaborn
sns.boxplot(x='Group', y='Value', data=df_plot)

# Add labels and title
plt.xlabel('Group')
plt.ylabel('Value')
plt.title('Distribution of the number of clusters')

# Show the plot
plt.show()

In [ ]:
clustering_stability = {}
for i in range(len(clustering_list)):
    clustering_stability[clustering_list[i]] = compute_rand_score(all_clustering[clustering_list[i]])

In [ ]:
# Combine the data in a single dataframe
df_plot = pd.DataFrame({'Group': [c for c in clustering_list for s in clustering_stability[c]],
                   'Value': [s for c in clustering_list for s in clustering_stability[c]]})

# Create a boxplot with Seaborn
sns.boxplot(x='Group', y='Value', data=df_plot)

# Add labels and title
plt.xlabel('Group')
plt.ylabel('Value')
plt.title('Distribution of the number of clusters')

# Show the plot
plt.show()

## Markowitz

In [ ]:
def find_representatives(clusters):
    nb_clustering = len(clusters.keys())
    representatives = []
    for i in range(nb_clustering):
        
        current_representatives = []
        current_cluster = clusters[list(clusters.keys())[i]]

        # Loop through each community
        for group in range(max(current_cluster.values())):
            subset = [cluster[0] for cluster in current_cluster.items() if cluster[1] == group]
            # Subset the data for the current group
            subset_data = df[subset]

            # Apply PCA
            pca = PCA()
            principal_components = pca.fit_transform(subset_data)

            # Identify the leading coin (biggest contributor in the first principal component)
            leading_coin_index = np.argmax(np.abs(pca.components_[0]))
            leading_coin = subset_data.columns[leading_coin_index]
            current_representatives.append(leading_coin)
            # Print
        representatives.append(current_representatives)
    return representatives

In [ ]:
def out_sample_risks(clusters, representatives):

    nb_clustering = len(clusters.keys())
    out_vol = []
    vol_abs = []
    for i in range(nb_clustering):
        length_window = list(clusters.keys())[i][1] - list(clusters.keys())[i][0]

        corr_mat_insample = df.loc[list(clusters.keys())[i][0]:list(clusters.keys())[i][1], representatives[i]].corr()
        corr_mat_outsample = df.loc[list(clusters.keys())[i][1]:list(clusters.keys())[i][1] + length_window, representatives[i]].corr()

        inv_corr_insample = LA.inv(corr_mat_insample.to_numpy())
        w_opt = inv_corr_insample @ np.ones(len(inv_corr_insample)) / (np.ones(len(inv_corr_insample)) @ inv_corr_insample @ np.ones(len(inv_corr_insample)))

        realized_vol = w_opt @ corr_mat_insample @ w_opt.T
        out_sample_vol = w_opt @ corr_mat_outsample @ w_opt.T
        out_vol.append(out_sample_vol)
        vol_abs.append(abs(realized_vol-out_sample_vol)/realized_vol)
    return (out_vol, vol_abs)

In [ ]:
df = pd.read_csv("../data/processed/normalized_log_ret.csv")

In [ ]:
clustering_out_vol = {}
for i in range(len(clustering_list)):
    representatives = find_representatives(all_clustering[clustering_list[i]])
    clustering_out_vol[clustering_list[i]] = out_sample_risks(all_clustering[clustering_list[i]], representatives)

In [ ]:
# Combine the data in a single dataframe
df_plot = pd.DataFrame({'Group': [c for c in clustering_list for risk in clustering_out_vol[c][0]],
                   'Value': [s for c in clustering_list for s in clustering_out_vol[c][0]]})

# Create a boxplot with Seaborn
sns.boxplot(x='Group', y='Value', data=df_plot)

# Add labels and title
plt.xlabel('Group')
plt.ylabel('Value')
plt.title('Out-of-sample volatility')

# Show the plot
plt.show()

In [ ]:
# Combine the data in a single dataframe
df_plot = pd.DataFrame({'Group': [c for c in clustering_list for risk in clustering_out_vol[c][1]],
                   'Value': [s for c in clustering_list for s in clustering_out_vol[c][1]]})

# Create a boxplot with Seaborn
sns.boxplot(x='Group', y='Value', data=df_plot)

# Add labels and title
plt.xlabel('Group')
plt.ylabel('Value')
plt.title('Absolute difference volatility')

# Show the plot
plt.show()